In [1]:
import os
import sys
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

In [2]:
from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

In [6]:
@torch.no_grad()
def load_yolo_predictor(device=0, 
                        dnn=False, 
                        data="",
                        weights=[]):
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data)
    return model

@torch.no_grad()
def run(
       model,
       source="",
       imgsz=(1024, 1024), 
       conf_thres=0.25,
       iou_thres=0.4,
       max_det=1000,
       device=0, 
       view_img=False, 
       save_txt=False,
       save_conf=False,
       save_crop=False,
       nosave=False,
       classes=None,
       agnostic_nms=False,
       augment=False,
       visualize=False,
       update=False,
       project="/home/sentic/Documents/data/storage/Madu_stuff/expi1", 
       name="expi1",
       exist_ok=False,
       line_thickness=3,
       hide_labels=False,
       hide_conf=False,
       half=False,
       ):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    save_dir = Path(project) / name
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    imgsz = check_img_size(imgsz, s=stride)  # check image size
    
    dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
    bs = 1  # batch_size
    
    vid_path, vid_writer = [None] * bs, [None] * bs
    model.warmup(imgsz=(1, 3, *imgsz), half=half)  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
#         im = cv2.imread("/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png")
#         im = np.transpose(im, (2, 0, 1))
        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2
        
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3
        for i, det in enumerate(pred):  # per image
            seen += 1
            p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
            from matplotlib import pyplot as plt
            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()
                for *xyxy, conf, cls in reversed(det):
                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
            LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)    
                
                

In [7]:
weights = ["/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_01_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_02_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_03_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_01_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_02_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_03_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_01_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_02_SGD//weights/best.pt",
           "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_03_SGD//weights/best.pt"
          ]
data = "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/nodule01.yaml"
source = "/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png"
augment = True

In [8]:
model = load_yolo_predictor(data=data,
                           weights=weights)

YOLOv5 🚀 v6.0-192-g436ffc4 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11019MiB)

Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs


Ensemble created with ['/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_01_SGD//weights/best.pt', '/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_02_SGD//weights/best.pt', '/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_03_SGD//weights/best.pt', '/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_01_SGD//weights/best.pt', '/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_02_SGD//weights/best.pt', '/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_03_SGD//weights/best.pt', '/home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/NODULE_COMPETITION/YOLOV5/yolov5/runs/train/exp_1024_01_SGD//weights/

image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.066s)


In [10]:
run(model=model, 
    source=source,
    augment=augment)

image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.085s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.090s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.094s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.093s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.095s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.098s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.098s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.101s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.102s)


image 1/1 /home/sentic/Documents/data/storage/Madu_stuff/ALL_CODE/Madu/images/n1134.png: 1024x1024 Done. (1.104s)
